In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys


from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD
# sys.version

2024-07-21 11:18:10.297743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Loading the data

In [2]:
input_texts = []
target_texts = []

In [3]:
for line in open("../poetry_generator/data/robert_frost.txt"):
    line = line.rstrip()
    if not line:
        continue
        
    input_line = '<sos>' + line
    target_line = line + '<eos>'
    
    input_texts.append(input_line)
    target_texts.append(target_line)

In [4]:
all_lines = input_texts + target_texts
all_lines[:15]

['<sos>Two roads diverged in a yellow wood,',
 '<sos>And sorry I could not travel both',
 '<sos>And be one traveler, long I stood',
 '<sos>And looked down one as far as I could',
 '<sos>To where it bent in the undergrowth;',
 '<sos>Then took the other, as just as fair,',
 '<sos>And having perhaps the better claim',
 '<sos>Because it was grassy and wanted wear,',
 '<sos>Though as for that the passing there',
 '<sos>Had worn them really about the same,',
 '<sos>And both that morning equally lay',
 '<sos>In leaves no step had trodden black.',
 '<sos>Oh, I kept the first for another day!',
 '<sos>Yet knowing how way leads on to way',
 '<sos>I doubted if I should ever come back.']

In [5]:
len(all_lines)

2872

### Converting sentences into integers
#### Tokenizer 1) Splits into individual tokens/words 2) each word is converted into an integer index for mapping 

In [6]:
from keras.preprocessing.text import Tokenizer

MAX_VOCAB_SIZE = 3000
MAX_SEQUENCE_LENGTH = 100

In [7]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer.fit_on_texts(all_lines)
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

target_sequences[:15]

[[133, 571, 572, 7, 3, 573],
 [4, 574, 5, 66, 28, 984],
 [4, 24, 25, 985, 163, 5],
 [4, 181, 69, 25, 14, 137, 14, 5],
 [2, 39, 8, 986, 7, 1],
 [130, 197, 1, 575, 14, 73, 14],
 [4, 145, 305, 1, 260],
 [440, 8, 12, 576, 4, 244],
 [155, 14, 13, 11, 1, 577, 2280],
 [21, 579, 50, 415, 122, 1],
 [4, 153, 11, 987, 988, 2281],
 [7, 989, 44, 581, 21, 990, 2282],
 [941, 5, 261, 1, 262, 13, 323],
 [191, 992, 106, 80, 993, 15, 2],
 [5, 994, 29, 5, 126, 154, 86, 2283]]

In [8]:
print(input_sequences[:15])

max_seq_len_from_data = max(len(s) for s in input_sequences)
max_seq_len_from_data


[[413, 571, 572, 7, 3, 573, 808], [16, 574, 5, 66, 28, 984, 153], [16, 24, 25, 985, 163, 5, 258], [16, 181, 69, 25, 14, 137, 14, 5, 66], [59, 39, 8, 986, 7, 1, 2408], [259, 197, 1, 575, 14, 73, 14, 2409], [16, 145, 305, 1, 260, 2410], [2411, 8, 12, 576, 4, 244, 2412], [414, 14, 13, 11, 1, 577, 58], [578, 579, 50, 415, 122, 1, 809], [16, 153, 11, 987, 988, 580], [102, 989, 44, 581, 21, 990, 991], [2413, 5, 261, 1, 262, 13, 323, 2414], [810, 992, 106, 80, 993, 15, 2, 80], [30, 994, 29, 5, 126, 154, 86, 324]]


11